# Predykcja wytrzymałości na ściskanie na podstawie rzeczywistych danych wpisanych

Za pomocą bibloteki "pickle" model został wyciągnięty oraz zserializowany w celu dalszego użytkowania. Dużym plusem jest to że nie potrzeba instalować dodatkowych bibliotek oprócz standardowej biblioteki "pickle"


Uruchamiaj po kolei poszczególne kolumny aby móc edytować wartości w pliku Excel

In [213]:
import pickle
import pandas as pd
import xlwings as xw
import numpy as np

W tym kroku przypiszemy wartości poszczególnych składników mieszaki betonowej

In [214]:
rows = [430,0, 0, 178.5, 8.0, 1001.9, 688.7, 28]

In [215]:
columns = ['cement [kg/m3]',"żużel_wielkopiecowy [kg/m3]",
       'popiół_lotny [kg/m3]', 'woda [kg/m3]',
       'superplastyfikator [kg/m3]', 'kruszywo_grube [kg/m3]',
       'kruszywo_drobne [kg/m3]', 'wiek [dni]']

In [216]:
predicted = pd.DataFrame([dict(zip(columns,rows))])
predicted

,cement [kg/m3],żużel_wielkopiecowy [kg/m3],popiół_lotny [kg/m3],woda [kg/m3],superplastyfikator [kg/m3],kruszywo_grube [kg/m3],kruszywo_drobne [kg/m3],wiek [dni]
0,430,0,0,178.5,8.0,1001.9,688.7,28


Wyświetlanie danych w excelu.

In [217]:
xw.view(predicted)

## Otwieramy teraz zapisane wcześniej modele

Model oparty na drzewach decyzyjnych

In [295]:
with open (r"modelPickle", "rb") as model_dat:
    model = pickle.load(model_dat)

Model wykorzystujący strukturę wzmacniającą gradient

In [296]:
with open (r"modelPickle2", "rb") as model_dat2:
    model2 = pickle.load(model_dat2)

I podstawiamy nasze przykładowe dane do wytrenowanego modelu

In [220]:
print(model.predict(predicted)[0], "MPa") #Drzewo decyzyjne

52.3 MPa


In [221]:
print(model2.predict(predicted)[0], "MPa") #Gradient boost

51.195663093324114 MPa


## Zedytuj teraz plik Excel podstawiając własne dane :)

Możesz teraz zedytować swoje wartości w otwartym pliku Excel. 

Po edycji wykonaj poniższą komórkę


In [222]:
predicted2 = xw.books.active.sheets[0].range("A1:I2").options(pd.DataFrame).value
predicted2


,cement [kg/m3],żużel_wielkopiecowy [kg/m3],popiół_lotny [kg/m3],woda [kg/m3],superplastyfikator [kg/m3],kruszywo_grube [kg/m3],kruszywo_drobne [kg/m3],wiek [dni]
,,,,,,,,
0.0,430.0,0.0,0.0,178.5,8.0,1001.9,688.7,28.0


In [223]:
wynik_dt = model.predict(predicted2)[0] #Drzewo decyzyjne

print(wynik_dt, "MPa")

52.3 MPa


In [224]:
wynik_Xboost = model2.predict(predicted2)[0] #Gradient boost

print(wynik_Xboost, "MPa")

51.195663093324114 MPa


Wstawimy teraz zapisane dane do Excela

In [225]:
ws = xw.books.active.sheets[0]

wiek = predicted2.loc[0,"wiek [dni]"]

ws["B4"].value = [
    ["Przewidziana wytrzymałość na ściskanie po {:.0f} dniach na podstawie:".format(wiek)],
    ["Drzewa decyzjnego"], 
    ["Algorytmu wzmacniającego strukturę gradientu"]
    ]

ws["B4"].columns.autofit()

ws["C5"].value = [
    ["{:.2f} MPa".format(wynik_dt)],
    ["{:.2f} MPa".format(wynik_Xboost)]
    ]

In [266]:
try:
    ws = xw.books.active.sheets[0]
except:
    xw.view(predicted)
    ws = xw.books.active.sheets[0]

## Jeżeli chcesz wykonać obliczenia dla innego zestawu danych to nie zamykaj pliku excel tylko wpisz ponowne wartości i uruchom ponownie poniższą komórkę

In [292]:
przyrost_dt = []
przyrost_xb = []

own_predict = ws["A1:I2"].options(pd.DataFrame).value

wynik_dt = model.predict(own_predict)[0] #Drzewo decyzyjne
wynik_Xboost = model2.predict(own_predict)[0] #Gradient boost

wiek = own_predict.loc[0,"wiek [dni]"]

ws["B4"].value = [
    ["Przewidziana wytrzymałość na ściskanie po {:.0f} dniach na podstawie:".format(wiek)],
    ["Drzewa decyzjnego"], 
    ["Algorytmu wzmacniającego strukturę gradientu"]
    ]

ws["B4"].columns.autofit()

ws["C5"].value = [
    ["{:.2f} MPa".format(wynik_dt)],
    ["{:.2f} MPa".format(wynik_Xboost)]
    ]

for i in range(1,29):
    own_predict.loc[0,"wiek [dni]"] = i

    wynik_dt = model.predict(own_predict)[0] #Drzewo decyzyjne
    wynik_Xboost = model2.predict(own_predict)[0] #Gradient boost

    przyrost_dt.append(wynik_dt)
    przyrost_xb.append(wynik_Xboost)


tbl = pd.DataFrame(list(zip(przyrost_dt,przyrost_xb)), columns=["Przyrost drzewo decyzyjne","Przyrost xBoost"], index=np.arange(1, len(przyrost_dt)+1))

Dodanie osobnego arkusza przedstawiającego przyrost wytrzymałości w czasie

In [293]:
if ws.book.sheets.count < 2:
    ws2 = ws.book.sheets.add(after=ws)
    
ws2["A1"].value = "Przyrost wytrzymałości na ściskanie w czasie"
ws2["A3"].expand().clear()
ws2["A1"].font.bold = True
ws2["A2"].value = tbl
ws2["B2:C2"].columns.autofit()


Dodanie wykresu ilustrującego wykres przyrostu wytrzymałości w czasie

In [294]:
if ws2.charts.count > 0:
    ws2.charts[0].delete()
    
chart = ws2.charts.add(ws2["D2"].left,ws2["D2"].top, width=620, height=250)
chart.set_source_data(ws2["A2"].expand())
chart.chart_type = 'line'

## Na podstawie danych możemy stwierdzić że algorytm uczenia maszynowego oparty na gradient boostingu daje najbardziej rzeczywiste rezultaty

Zachęcam do testowania na własnych rzeczywistych danych
